# Data Preparation For Fine Grained Attributes 

In [35]:
import pandas as pd 
import numpy as np 

paths_df = pd.read_csv("../datasets/attribute_set/train_fine_grained.txt", names=["paths"], sep=" ", header=None) 

In [36]:
paths_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14000 entries, 0 to 13999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   paths   14000 non-null  object
dtypes: object(1)
memory usage: 109.5+ KB


In [37]:
columns = pd.read_csv("../datasets/attribute_set/fine_grained_column_names.txt", header=None, delimiter=" ").to_numpy().squeeze()

In [38]:
attr_df = pd.read_csv("../datasets/attribute_set/train_attr_fine_grained.txt", header=None, delimiter=" ").iloc[:, :-1]
attr_df.columns = columns
attr_df.head() 

,floral,graphic,striped,embroidered,pleated,solid,lattice,long_sleeve,short_sleeve,sleeveless,...,no_neckline,denim,chiffon,cotton,leather,faux,knit,tight,loose,conventional
0,0,0,0,1,0,0,0,0,0,1,...,1,0,1,0,0,0,0,0,0,1
1,0,0,0,0,0,1,0,0,0,1,...,1,0,0,1,0,0,0,1,0,0
2,0,1,0,0,0,0,0,0,0,1,...,1,0,0,1,0,0,0,0,0,1
3,0,0,0,0,0,1,0,0,0,1,...,1,1,0,0,0,0,0,1,0,0
4,0,0,0,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1


In [40]:
attr_df.shape, paths_df.shape 

((14000, 26), (14000, 1))

In [41]:
base = "../datasets/big_ds/img-001/" 
base

'../datasets/big_ds/img-001/'

In [42]:
paths_df = paths_df.apply(lambda x: base + x) 
paths_df.head() 

,paths
0,../datasets/big_ds/img-001/img/Sweet_Crochet_B...
1,../datasets/big_ds/img-001/img/Classic_Pencil_...
2,../datasets/big_ds/img-001/img/Strapless_Diamo...
3,../datasets/big_ds/img-001/img/Mid-Rise_-_Acid...
4,../datasets/big_ds/img-001/img/Zippered_Single...


In [45]:
data = pd.concat([paths_df, attr_df], axis=1)
data.head() 

,paths,floral,graphic,striped,embroidered,pleated,solid,lattice,long_sleeve,short_sleeve,...,no_neckline,denim,chiffon,cotton,leather,faux,knit,tight,loose,conventional
0,../datasets/big_ds/img-001/img/Sweet_Crochet_B...,0,0,0,1,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,1
1,../datasets/big_ds/img-001/img/Classic_Pencil_...,0,0,0,0,0,1,0,0,0,...,1,0,0,1,0,0,0,1,0,0
2,../datasets/big_ds/img-001/img/Strapless_Diamo...,0,1,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,1
3,../datasets/big_ds/img-001/img/Mid-Rise_-_Acid...,0,0,0,0,0,1,0,0,0,...,1,1,0,0,0,0,0,1,0,0
4,../datasets/big_ds/img-001/img/Zippered_Single...,0,0,0,0,0,1,0,1,0,...,0,0,0,1,0,0,0,0,0,1


# Prepare Data Pipeline by using tf.data 

In [46]:
fnames = data.paths.to_numpy()  
fnames[:5]

array(['../datasets/big_ds/img-001/img/Sweet_Crochet_Blouse/img_00000070.jpg',
       '../datasets/big_ds/img-001/img/Classic_Pencil_Skirt/img_00000010.jpg',
       '../datasets/big_ds/img-001/img/Strapless_Diamond_Print_Dress/img_00000038.jpg',
       '../datasets/big_ds/img-001/img/Mid-Rise_-_Acid_Wash_Skinny_Jeans/img_00000010.jpg',
       '../datasets/big_ds/img-001/img/Zippered_Single-Button_Blazer/img_00000078.jpg'],
      dtype=object)

In [48]:
import tensorflow as tf 

ds_size = data.shape[0] 
number_of_selected_samples = 2000 

# filelist_ds = tf.data.Dataset.from_tensor_slices(fnames[:number_of_selected_samples]) 
filelist_ds = tf.data.Dataset.from_tensor_slices(fnames) 


filelist_ds.cardinality().numpy() 

14000

## Custom tf Helpers 

In [49]:
def get_label(file_path):
    """
        file_path: the file path for the image that you want to select
    """
    labels = data.loc[data.paths == file_path].to_numpy().squeeze()[1:].astype("int64")
    return tf.convert_to_tensor(labels) 

In [50]:
get_label(fnames[0])

<tf.Tensor: shape=(26,), dtype=int64, numpy=
array([0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1], dtype=int64)>

In [51]:
# resize and scale the images so that we can save time in training  
IMG_WIDTH, IMG_HEIGHT = 64, 64 
def decode_img(img):
    """
        img: img is the image 
    """ 
    #color images 
    img = tf.image.decode_jpeg(img, channels=3) 
    img = tf.image.convert_image_dtype(img, tf.float32) 
    img = tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT]) 
    img = img / tf.constant(256, dtype=tf.float32)
    return img

In [52]:
def combine_images_labels(file_path: tf.Tensor): 
    label = get_label(file_path) 
    img = tf.io.read_file(file_path) 
    img = decode_img(img) 
    return img, label 

In [53]:
ds_train = filelist_ds 

In [54]:
BATCH_SIZE = 32 

## Pre-process All the Images 

In [56]:
ds_train = ds_train.map(lambda x: 
                        tf.py_function(func=combine_images_labels, 
                                       inp=[x], # input of the function 
                                       Tout=(tf.float32,tf.int64)),  # return type 
                        num_parallel_calls=tf.data.AUTOTUNE, # parallelizing data extraction 
                        deterministic=False 
                        )

TypeError: in user code:


    TypeError: <lambda>() takes 1 positional argument but 2 were given


### Prepare Data Pipeline 

- **batch**(): Combines consecutive elements of this dataset into batches.
- **cache**(): Caches the elements in this dataset. he first time the dataset is iterated over, its elements will be cached either in the specified file or in memory.Subsequent iterations will use the cached data.
- **prefetch**(): Creates a Dataset that prefetches elements from this dataset. Most dataset input pipelines should end with a call to *prefetch*. This allows later elements to be prepared while the current element is being processed. This often improves latency and throughput, at the cost of using additional memory to store prefetched elements.
 

In [57]:
ds_train_batched = ds_train.batch(BATCH_SIZE).cache().prefetch(tf.data.experimental.AUTOTUNE) 

In [58]:
ds_train_batched.cardinality().numpy() 

438

In [59]:
32 * 438

14016

## Create a Keras CNN model by using Transfer learning

In [63]:
from tensorflow import keras 
base_model = keras.applications.VGG16(
    weights="imagenet", # load weights pre-trained on ImageNet. 
    input_shape=(IMG_WIDTH, IMG_HEIGHT, 3), # VGG16 expects min 32 x 32 
    include_top = False # do not include output layer of the image net vgg 
)
base_model.trainable = False 

In [60]:
nr_of_classes = len(columns) 
nr_of_classes

26

In [64]:
inputs = keras.Input(shape=(IMG_WIDTH,IMG_HEIGHT,3)) 
x = base_model(inputs) 
x = keras.layers.GlobalAveragePooling2D()(x) 

initializer = tf.keras.initializers.GlorotUniform(seed=42) 
activation = tf.keras.activations.sigmoid  

outputs = keras.layers.Dense(nr_of_classes,
                             kernel_initializer=initializer, 
                             activation=activation)(x) 

model = keras.Model(inputs, outputs) 

## Compile and Train the Model 

In [65]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(), # default from_logits=False
              metrics=[keras.metrics.BinaryAccuracy()])

In [66]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3) 
checkpoint_path = "checkpoints/attribute_prediction_classifier_fine_grained/checkpoint.ckpt" 
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                         save_weights_only=True,
                                                         save_best_only=True,
                                                         verbose=1)

In [67]:
model.fit(ds_train_batched, 
        epochs=10,
        callbacks=[early_stopping, checkpoint_callback])

Epoch 1/10
